# 1212. Team Scores in Football Tournament
[問題](https://leetcode.com/problems/team-scores-in-football-tournament/description/?envType=study-plan-v2&envId=premium-sql-50)

## 解答の方針

勝ち点の判定を行うメソッド`compute`を作る。  
matchesテーブルに`host_point`と`guest_point`の列を作る。  



In [1]:
import pandas as pd

In [139]:
data = [[10, 'Leetcode FC'], [20, 'NewYork FC'], [30, 'Atlanta FC'], [40, 'Chicago FC'], [50, 'Toronto FC']]
teams = pd.DataFrame(data, columns=['team_id', 'team_name']).astype({'team_id':'Int64', 'team_name':'object'})
data = [[1, 10, 20, 3, 0], [2, 30, 10, 2, 2], [3, 10, 50, 5, 1], [4, 20, 30, 1, 0], [5, 50, 30, 1, 0]]
matches = pd.DataFrame(data, columns=['match_id', 'host_team', 'guest_team', 'host_goals', 'guest_goals']).astype({'match_id':'Int64', 'host_team':'Int64', 'guest_team':'Int64', 'host_goals':'Int64', 'guest_goals':'Int64'})

In [74]:
teams = """
| team_id | team_name   |
| 10      | Leetcode FC |
| 20      | NewYork FC  |
| 30      | Atlanta FC  |
"""

In [75]:
matches = """
| match_id | host_team | guest_team | host_goals | guest_goals |
| 1        | 10        | 20         | 0          | 0           |
| 2        | 10        | 30         | 2          | 2           |
| 3        | 20        | 30         | 1          | 1           |
"""

In [76]:
teams = teams.replace(" ", "")
matches = matches.replace(" ", "")
with open("data.txt", "w") as f:
    f.write(teams)
teams = pd.read_csv("data.txt", sep="|")
teams = teams.dropna(axis=1)

with open("data.txt", "w") as f:
    f.write(matches)
matches = pd.read_csv("data.txt", sep="|")
matches = matches.dropna(axis=1)



In [77]:
teams

,team_id,team_name
0,10,LeetcodeFC
1,20,NewYorkFC
2,30,AtlantaFC


In [78]:
matches

,match_id,host_team,guest_team,host_goals,guest_goals
0,1,10,20,0,0
1,2,10,30,2,2
2,3,20,30,1,1


In [140]:
from typing import Optional
def compute(host_goal: int, guest_goal: int) -> Optional[int]:
    ans = None
    if host_goal == guest_goal:
        ans = 1
    elif host_goal > guest_goal:
        ans = 3
    elif host_goal < guest_goal:
        ans = 0
    return ans

In [143]:
def team_scores(teams: pd.DataFrame, matches: pd.DataFrame) -> pd.DataFrame:
    matches["host_point"] = list(map(compute, matches["host_goals"], matches["guest_goals"]))
    matches["guest_point"] = list(map(compute, matches["guest_goals"], matches["host_goals"]))
    group_host = matches.groupby("host_team")["host_point"].sum().reset_index()
    group_guest = matches.groupby("guest_team")["guest_point"].sum().reset_index()
    group_host.rename({"host_team": "team_id", "host_point": "points"}, axis=1, inplace=True)
    group_guest.rename({"guest_team": "team_id", "guest_point": "points"}, axis=1, inplace=True)
    points = pd.concat([group_host, group_guest], join="inner")
    num_points = points.groupby("team_id").sum().reset_index()
    num_points.rename({"points": "num_points"}, axis=1, inplace=True)
    result = teams.merge(num_points, left_on="team_id", right_on="team_id", how="outer")
    result["num_points"] = result["num_points"].fillna(0)
    return result.sort_values(by=["num_points", "team_id"], ascending=[False, True])

In [144]:
team_scores(teams, matches)

,team_id,team_name,num_points
0,10,Leetcode FC,7.0
1,20,NewYork FC,3.0
4,50,Toronto FC,3.0
2,30,Atlanta FC,1.0
3,40,Chicago FC,0.0


## Test scripts

In [82]:
from typing import Optional
def compute(host_goal: int, guest_goal: int) -> Optional[int]:
    ans = None
    if host_goal == guest_goal:
        ans = 1
    elif host_goal > guest_goal:
        ans = 3
    elif host_goal < guest_goal:
        ans = 0
    return ans

In [83]:
matches["host_point"] = list(map(compute, matches["host_goals"], matches["guest_goals"]))

In [84]:
matches["guest_point"] = list(map(compute, matches["guest_goals"], matches["host_goals"]))

In [85]:
matches

,match_id,host_team,guest_team,host_goals,guest_goals,host_point,guest_point
0,1,10,20,0,0,1,1
1,2,10,30,2,2,1,1
2,3,20,30,1,1,1,1


In [124]:
group_host = matches.groupby("host_team")["host_point"].sum().reset_index()

In [125]:
group_guest = matches.groupby("guest_team")["guest_point"].sum().reset_index()

In [126]:
group_host.rename({"host_team": "team_id", "host_point": "points"}, axis=1, inplace=True)

In [127]:
group_guest.rename({"guest_team": "team_id", "guest_point": "points"}, axis=1, inplace=True)

In [128]:
points = pd.concat([group_host, group_guest], join="inner")

In [129]:
points

,team_id,points
0,10,2
1,20,1
0,20,1
1,30,2


In [109]:
num_points = points.groupby("team_id").sum().reset_index()

In [117]:
num_points.rename({"points": "num_points"}, axis=1, inplace=True)

In [118]:
result = teams.merge(num_points, left_on="team_id", right_on="team_id", how="outer")

In [120]:
result

,team_id,team_name,num_points
0,10,LeetcodeFC,2
1,20,NewYorkFC,2
2,30,AtlantaFC,2


In [121]:
result["num_points"] = result["num_points"].fillna(0)

In [122]:
result[["team_id", "team_name" ,"num_points"]].sort_values(by="num_points", ascending=False)

,team_id,team_name,num_points
0,10,LeetcodeFC,2
1,20,NewYorkFC,2
2,30,AtlantaFC,2
